# Charts

In [1]:
# default_exp charts
# export
import altair as alt
from altair import Chart
alt = alt
Chart = Chart
from torch_snippets.loader import *

torch-snippets is preinstalling altair and exposing it's Chart class

In [2]:
from torch_snippets.loader import *
from sklearn.datasets import make_moons

x, y = make_moons(1000, noise=0.1)
df = pd.DataFrame({'x1': x[:,0], 'x2': x[:,1], 'y': y})

Chart(df).mark_circle().encode(
    x='x1:Q',
    y='x2:Q',
    color='y:N'
).interactive()


alt.Chart(...)

Refer to [altair-viz.github.io](altair-viz.github.io) for more awesome charts

In [46]:
# export
def confusion_matrix(df=None, truth=None, pred=None, threshold=None, mapping=None):
    if df is None:
        df = pd.DataFrame({'truth': truth, 'pred': pred})
        truth = 'truth'
        pred = 'pred'
    if threshold is None:
        threshold = len(df) // 4
    if mapping:
        assert isinstance(mapping, dict), 'mapping should be a dictionary'
        df[truth] = df[truth].map(lambda x: mapping[x])
        df[pred] = df[pred].map(lambda x: mapping[x])
    base = Chart(df).transform_aggregate(
        num_vals='count()',
        groupby=[truth, pred]
    ).encode(
        alt.X(f'{truth}:O', scale=alt.Scale(paddingInner=0)),
        alt.Y(f'{pred}:O', scale=alt.Scale(paddingInner=0)),
    )

    hm = base.mark_rect().encode(
        color=alt.Color('num_vals:Q',
            legend=None # alt.Legend(direction='horizontal')
        )
    )

    tx = base.mark_text(baseline='middle').encode(
        text='num_vals:Q',
        color=alt.condition(
            alt.datum.num_vals > threshold, 
            alt.value('white'), 
            alt.value('black'))
    )
    
    try:
        from sklearn.metrics import classification_report
        print(classification_report(df['truth'], df['pred']))
    except:
        ...

    return hm + tx

CM = confusion_matrix

### Method 1

In [47]:
truth=np.random.randint(2, size=100)
pred=np.random.randint(2, size=100)
show(CM(truth=truth, pred=pred, mapping={0: 'survived', 1: 'dead'}))
# mapping is optional

              precision    recall  f1-score   support

        dead       0.40      0.43      0.42        44
    survived       0.53      0.50      0.51        56

    accuracy                           0.47       100
   macro avg       0.47      0.47      0.47       100
weighted avg       0.47      0.47      0.47       100



alt.LayerChart(...)

### Method 2

In [49]:
df = pd.DataFrame({'truth': [randint(2) for _ in range(100)], 'pred': [randint(2) for _ in range(100)]})
show(CM(df, 'truth', 'pred', mapping={0: 'No', 1: 'Yes', 2: 'Na'}))
# mapping is optional

              precision    recall  f1-score   support

          No       0.54      0.59      0.56        49
         Yes       0.57      0.51      0.54        51

    accuracy                           0.55       100
   macro avg       0.55      0.55      0.55       100
weighted avg       0.55      0.55      0.55       100



alt.LayerChart(...)